# MINERÍA DE TEXTOS

## Alumna: Jessica Sarahi Méndez Rincón


###  Tarea 6
### Clase 5 Tarea 1

# NLTK

## Artículo de Upendra Sapkota and Steven Bethard and Manuel Montes-y-Gómez 
# Not All Character N-grams Are Created Equal: A Study in Authorship Attribution
 ### No todos los N-gramas de caracteres son iguales: un estudio en Atribución de Autoría (AA)

In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [2]:
np.random.seed(500)

### Con referencia al artículo...

Los n-gramas de caracteres son los más exitosos.
Característica en la atribución de autoría (Koppel et al., 2009; Frantzeskou et al., 2007; Koppel et al., 2011), pero el
La razón de su éxito no se entiende bien. Uno La hipótesis es que los n-gramos de caracteres llevan un poco de todo: contenido léxico, contenido sintáctico y
estilo uniforme mediante puntuación y espacios en blanco.


Comprensión de principios de los ngrams de caracteres que informará su uso como características para AA y otras tareas; por otro lado queremos hacer
Los enfoques de Atribución de Autoría (AA) son más accesibles para los no expertos.

En el artículo indican 3 tipos diferentes:


    1.- Morfo-sintaxis (como se representa por n-gramas tipo afijo)     
    
    2.- Contenido temático (como se representa por n-gramos como palabras)     
    
    3.- Estilo (como se representa por n-gramas basados en puntuación)  
    


En el artículo usaron los DataSet de CCAT 10 y CCAT 50 pertenecen a la categoría CCAT (sobre noticias corporativas / industriales) y están equilibradas
entre autores, con 100 documentos muestra para cada autor. Inspección manual del conjunto de datos revelado que algunos de los autores en esta colección constantemente
firmas usadas al final de los documentos. También,notamos que algunos escritores usan muchas citas.


Capacitamos a los clasificadores de máquinas de vectores de soporte (SVM) utilizando la implementación de Weka.

Algunos experimentos comparativos con la implementación de Weka de  clasificadores ingenuos de Bayes y el Lib- Implementación SVM de SVM.

De acuerdo a lo anterior puedo pensar que uno de los objetivos a la par es usar los algoritmos de analisis para clasificar correctamente a los textos para identificar a que autor corresponden.

### Aplicación y Desarrrollo.

Para ello desarrollaré en base a un dataset sencillo que me permita realizar la implementación, el desarrollo y la implementación de los Algoritmos de Naive Bayes y de SVM.

Con respecto al dataset es un ejemplo de texto etiquetado, debido a que no conseguí los Dataset que indican en el artículo.

###### Agregar los datos usando pandas

In [16]:
# Agregar los datos usando pandas
Corpus = pd.read_csv('corpus.csv', sep='\t', header=None,names=['target', 'text' ])

In [17]:
Corpus.head(100)

,target,text
0,__label__2,Stuning even for the non-gamer: This sound tra...
1,__label__2,The best soundtrack ever to anything.: I'm rea...
2,__label__2,Amazing!: This soundtrack is my favorite music...
3,__label__2,Excellent Soundtrack: I truly like this soundt...
4,__label__2,"Remember, Pull Your Jaw Off The Floor After He..."
...,...,...
95,__label__1,Very Not Worth Your Time: The book was wriiten...
96,__label__2,"Very fun and educational: Trains, shapes and p..."
97,__label__1,Ludicrous and silly: I remember getting this b...
98,__label__2,Artistry: I think that the Deodato concerts ar...


In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jess\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

###### Paso - 1: Preprocesamiento de datos: esto ayudará a obtener mejores resultados a través de los algoritmos de clasificación
 

In [19]:
# Paso - 1a: elimine las filas en blanco si las hay.
Corpus['text'].dropna(inplace=True)
# Paso - 1b: Cambia todo el texto a minúsculas. Esto es necesario ya que Python interpreta 'perro' y 'PERRO' de manera diferente
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
#Corpus['text'] = Corpus.apply(lambda x: x.astype(str).str.lower())
#data = [line.strip() for line in open("C:\corpus\TermList.txt", 'r')]
#texts = [[word.lower() for word in text.split()] for text in data]

Corpus.head(5)


,target,text
0,__label__2,stuning even for the non-gamer: this sound tra...
1,__label__2,the best soundtrack ever to anything.: i'm rea...
2,__label__2,amazing!: this soundtrack is my favorite music...
3,__label__2,excellent soundtrack: i truly like this soundt...
4,__label__2,"remember, pull your jaw off the floor after he..."


In [20]:

# Paso - 1c: Tokenización: en este cada entrada en el corpus se dividirá en un conjunto de palabras
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]

# Paso - 1d: Eliminar palabras de detención, no numéricas y realizar el vástago / lemmenting de palabras.
# WordNetLemmatizer requiere etiquetas Pos para comprender si la palabra es sustantivo o verbo o adjetivo, etc. 
# De manera predeterminada, está configurada como Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
     # Declarar la lista vacía para almacenar las palabras que siguen las reglas de este paso
    Final_words = []
    # Inicializando WordNetLemmatizer ()
    word_Lemmatized = WordNetLemmatizer()
    # La función pos_tag a continuación proporcionará la 'etiqueta', es decir, si la palabra es Sustantivo (N) o Verbo (V) u otra cosa.
    for word, tag in pos_tag(entry):
        # La siguiente condición es verificar las palabras Stop y considerar solo alfabetos
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # El conjunto final procesado de palabras para cada iteración se almacenará en 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)
print(Corpus.loc[index,'text_final'])

['beautiful', 'pen', 'fast', 'delivery', 'pen', 'ship', 'promptly', 'classic', 'montblanc', 'pen', 'everyone', 'rave', 'black', 'color', 'golden', 'trim', 'hold', 'thankful', 'parent', 'gift', 'graduation', 'present', 'someday', 'plan', 'buy', 'pen', 'vendor', 'amazon', 'amaze', 'long', 'time', 'shopper', 'amazon', 'mostly', 'pen', 'electronics', 'however', 'pen', 'gift', 'mom', 'nothing', 'good', 'thing', 'say', 'part', 'love']


In [21]:
Corpus.head(5)

,target,text,text_final
0,__label__2,"[stuning, even, for, the, non-gamer, :, this, ...","['stun', 'even', 'sound', 'track', 'beautiful'..."
1,__label__2,"[the, best, soundtrack, ever, to, anything, .,...","['best', 'soundtrack', 'ever', 'anything', 're..."
2,__label__2,"[amazing, !, :, this, soundtrack, is, my, favo...","['amaze', 'soundtrack', 'favorite', 'music', '..."
3,__label__2,"[excellent, soundtrack, :, i, truly, like, thi...","['excellent', 'soundtrack', 'truly', 'like', '..."
4,__label__2,"[remember, ,, pull, your, jaw, off, the, floor...","['remember', 'pull', 'jaw', 'floor', 'hear', '..."


###### Paso - 2: dividir el modelo en el conjunto de datos de entrenamiento y prueba

In [22]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['target'],test_size=0.3)

###### Paso - 3: la etiqueta codifica la variable objetivo - Esto se hace para transformar los datos categóricos del tipo de cadena en el conjunto de datos en valores numéricos

In [23]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [24]:
Corpus.head


<bound method NDFrame.head of            target                                               text  \
0     __label__2   [stuning, even, for, the, non-gamer, :, this, ...   
1     __label__2   [the, best, soundtrack, ever, to, anything, .,...   
2     __label__2   [amazing, !, :, this, soundtrack, is, my, favo...   
3     __label__2   [excellent, soundtrack, :, i, truly, like, thi...   
4     __label__2   [remember, ,, pull, your, jaw, off, the, floor...   
...           ...                                                ...   
9989  __label__2   [a, revelation, of, life, in, small, town, ame...   
9990  __label__2   [great, biography, of, a, very, interesting, j...   
9991  __label__1   [interesting, subject, ;, poor, presentation, ...   
9992  __label__1   [do, n't, buy, :, the, box, looked, used, and,...   
9993  __label__2   [beautiful, pen, and, fast, delivery, ., :, th...   

                                             text_final  
0     ['stun', 'even', 'sound', 'track', 'beaut

###### Paso - 4: Vectoriza las palabras usando TF-IDF Vectorizer - Esto se hace para encontrar la importancia de una palabra en el documento en comparación para el corpus

In [25]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [26]:
print(Tfidf_vect.vocabulary_)

{'stun': 4279, 'even': 1520, 'sound': 4131, 'track': 4556, 'beautiful': 380, 'paint': 3156, 'mind': 2831, 'well': 4867, 'would': 4952, 'recomend': 3599, 'people': 3225, 'hate': 2047, 'game': 1860, 'music': 2920, 'play': 3296, 'cross': 1023, 'ever': 1524, 'best': 420, 'back': 324, 'away': 314, 'crude': 1026, 'take': 4370, 'fresh': 1818, 'step': 4219, 'guitar': 2001, 'soulful': 4128, 'orchestra': 3092, 'impress': 2238, 'anyone': 201, 'care': 638, 'listen': 2607, 'soundtrack': 4133, 'anything': 202, 'read': 3565, 'lot': 2650, 'review': 3723, 'say': 3844, 'figure': 1692, 'write': 4957, 'disagree': 1224, 'bit': 442, 'ultimate': 4633, 'masterpiece': 2746, 'timeless': 4501, 'year': 4976, 'beauty': 382, 'simply': 4020, 'refuse': 3625, 'price': 3408, 'tag': 4369, 'pretty': 3403, 'must': 2926, 'go': 1931, 'buy': 593, 'cd': 670, 'much': 2910, 'money': 2869, 'one': 3071, 'feel': 1669, 'worth': 4948, 'every': 1526, 'amaze': 149, 'favorite': 1658, 'time': 4500, 'hand': 2019, 'intense': 2326, 'sadnes

In [27]:
print(Train_X_Tfidf)

  (0, 4957)	0.07586733140979379
  (0, 4942)	0.09089372327775941
  (0, 4903)	0.3090049850489963
  (0, 4867)	0.06428363380327687
  (0, 4648)	0.14020777499195844
  (0, 4396)	0.14499159789872404
  (0, 4346)	0.16973233039759245
  (0, 4250)	0.12946682926754657
  (0, 4206)	0.08683718340340323
  (0, 4046)	0.14020777499195844
  (0, 3988)	0.10275103018015055
  (0, 3944)	0.10301312487725234
  (0, 3783)	0.17864121444541547
  (0, 3753)	0.15850846388039252
  (0, 3319)	0.09549495601432467
  (0, 3264)	0.10592198164914096
  (0, 3249)	0.16973233039759245
  (0, 3213)	0.1737383017534868
  (0, 3120)	0.1620793316801583
  (0, 3071)	0.04922790752729131
  (0, 3024)	0.16797363109825075
  (0, 2900)	0.14273607963769358
  (0, 2763)	0.10283804816150456
  (0, 2700)	0.05747795617866072
  (0, 2584)	0.053363462674831504
  :	:
  (6993, 2515)	0.1187411912477818
  (6993, 2118)	0.14090129285955816
  (6993, 1973)	0.0735317987507145
  (6993, 1779)	0.22657826251647156
  (6993, 1744)	0.20518488512814215
  (6993, 1707)	0.139427

###### Paso - 5: ahora podemos ejecutar diferentes algoritmos para clasificar la verificación de datos para la precisión

In [28]:
# Clasificador - Algoritmo - Naive Bayes
# Se ajusta el conjunto de datos de entrenamiento en el clasificador
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# Predecir las etiquetas en el conjunto de datos de validación
predictions_NB = Naive.predict(Test_X_Tfidf)
# Usando la función de precision_score para obtener la precisión
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  82.86095365121707


In [29]:
# Clasificador - Algoritmo - SVM
# Se ajusta el conjunto de datos de entrenamiento en el clasificador
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predecir las etiquetas en el conjunto de datos de validación
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Usando la función precision_score para obtener la precisión
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  83.09436478826275


# Preguntas establecidas en el artículo

En algunas preguntas si fueron algo coplejas debido a que en el artículo se enfocan en los resultados más que en compartir los algoritmos.

### Pregunta 1
1.- ¿Son todos los n-gramas de caracteres igualmente importantes? 

Se divide los n-gramas de caracteres en diez categorías disjuntas,
ilustrando empíricamente qué categorías existen. La precisión de
cada tipo de n-grama para cada uno de los diferentes dataset. En el artículo se muestra que las cuatro categorías principales para
los AA de dominio único son: prefijo, sufijo, prefijo espacial,y a infijo de palabra.

In [35]:
def respuesta_uno(text, n): 
    n-=1 
    return [text[i-n:i+1] for i,char in enumerate(text)][n:] 

In [38]:
respuesta_uno('Biblioteca',3)

['Bib', 'ibl', 'bli', 'lio', 'iot', 'ote', 'tec', 'eca']

Definitivamente tendrá el impacto en cada uno de los n-gramas que ayuden a completar o que tengan mayor impacto para la palabra completa.

### Pregunta 2

2.- Por ejemplo, son el prefijo de "there", el sufijo de "breathe" y toda la palabra "the" todo equivalente?


Respuesta:


En el artículo se menciona que el prefijo espacial son características fuertes, y generalmente son
mejor que las características del sufijo, quizás porque
los autores tienen más control sobre los prefijos en inglés,
mientras que los sufijos son a menudo obligatorios para la gramática
razones.

In [40]:
respuesta_uno('there',3)

['the', 'her', 'ere']

In [41]:
respuesta_uno('breathe',3)

['bre', 'rea', 'eat', 'ath', 'the']

In [42]:
respuesta_uno('the',3)

['the']

### Pregunta 3

3.- Más generalmente, son caracteres n-gramas que capturan información morfosintáctica, temática información e información de estilo por igual ¿importante?

Respuesta:

En el artículo indican que los experimentos  han demostrado, por ejemplo, que los prefijos n-gramas son universalmente predictivos

In [48]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

def respuesta_tres():
    vect = CountVectorizer().fit(Train_X)
    X_train_vectorized=vect.transform(Train_X)
    clf=MultinomialNB()
    clf.fit(X_train_vectorized, Train_Y)
    X_test_vectorized=vect.transform(Test_X)
    predictions = clf.predict(X_test_vectorized)
    print(predictions)
    return roc_auc_score(Test_Y,predictions)

In [49]:
respuesta_tres()

[0 0 0 ... 1 1 0]


0.815103765820673

El 81.51% indica que si son importantes de acuerdo al dataset.

### Pregunta 4
4.- ¿Son los caracteres n-gramas más importantes? 

Respuesta:

En el artículo se comenta que la configuración de dominio único, nuevamente se ve ese sufijo y
las palabras intermedias también son altamente predictivas, mientras que en las
configuraciones de dominio cruzado, nuevamente vemos que prefijo e infijo son altamente predictivos. Todos estos resultados
confirman que algunos tipos de n-gramas son fundamentalmente
más predictivo que otros.

De acuerdo a lo que revisé en la aplicación de Algoritmos la eficiencia es muy similar. Sin embargo depende del idioma y de la forma en como se plantee el texto.

### Pregunta 5

5.- Para configuraciones de dominio único también la mayoría importante para la configuración entre dominios? 

Respuesta:

El análisis va a nivel de anagramas de caracteres, ni siquiera a palbras. Por lo que la forma en que se comparen dichos anagramas solo ayuda para completar la palabra, no para hacer comparativa entre palabras. Lo cual hace un artículo muy rico en la proyección de su investigación, ya que al completar las palbras correctamente o con una probabilidad muy cercana, también puede ayudar al analisis de anagramas de palabras.

### Pregunta 6

6.- Cuales los n-gramas de caracteres son más como una bolsa de palabras  características (que tienden a rastrear temas) y cuáles son más como rasgos estilísticos (que tienden a rastrear autores)?


Respuesta:

Como ya se demostró en el artículo, las funciones puntuales funcionan mejor que usar todas las
características, nos gustaría utilizar.. De siete categorías en afijo y
puntual, se calculó en cuántos de ellos cada elemento le pertenece, siendo tres el máximo posible
valor.   Está claro que los caracteres más significativos son los que rodean la puntuación
de elemtnos y los espacios alrededor son segundos
más negativos, mientras que los más significativos o postivos elementos son los
que estan en medio de largas palabras. Esto nos da un
idea sobre los caracteres en un texto que son importantes
para tareas de AA.

### Pregunta 7

7.- ¿Están de acuerdo los diferentes clasificadores sobre la importancia? De los diferentes tipos de caracteres n-gramas? 

Respuesta:
    
Si, los clasificadores ayudan en la mayoría de los casos en ser similares, como lo vimos en la programación de algoritmos en líneas pasadas.

Naive Bayes Accuracy Score ->  82.86095365121707

SVM Accuracy Score ->  83.09436478826275

### Pregunta 8

8.- Son  algunos caracteres n-gramas son siempre los mejores independientemente del algoritmo de aprendizaje?

Respuesta:

Posiblemente sí, pero el impacto no es mucho, con los algoritmos que vimos establecimos una detección muy similar de los textos.


### Pregunta 9
9.- ¿Son irrelevantes algunos tipos de caracteres n-gramas?  En tareas de AA?


Respuesta:

Todo cuenta sobre todo para la predicción ya que conforman los elementos de palabras y a su vez de parrafos y textos completos.

### Pregunta 10


10.- ¿Hay categorías de elementos? n-gramas que podemos excluir y obtener similares (o mejor) rendimiento que usar todos los n-gramos?


Si los hay, ¿son iguales para ambos?


Respuesta:
    
Posiblemente aquellas palabras que están conformadas de pocas consonantes, eso ayudaría a esclarecer el como las palabras alternas se pueden conectar.


# Conclusión

Para concluir puedo comentar que el artículo tiene un excelente propuesta de análisis, y un noblre proposito de identificar autoria de textos incluso antiguos que ayuden a preservar el material literario de escritores importantes que dejaron su obra y no salio a luz en sus tiempos.

Sin embargo, cabe mencionar que como he identificado, en muchos artículos es complicado entrar a revisar los dataset.

En cuanto al código en muchas empresas de inteligencia artificial lo comparten en sus sitios web para compartirlo con la comunidad y que no solo lo prueben sino que lo puedan mejorar con propuestas novedosas, ya que eso lo retroalimentan de forma significativa atrayendo al colarboración de cientificos a nivel mundial interesados en Ciencia de Datos.

También es entendible que los artículos sean el resumen de los hallazgos y el informe de resultados que se produjeron.

Concluyo que es sin duda alguno un artículo muy interesante que nutre los conocimientos adquiridos en el curso.
Agradezco que lo hayamos abordado como tema en el curso ya que podemos ir descubriendo por nuestra cuenta artículos relacionados a Lenguaje Natural de otras fuentes que de igual forma nos enriquescan como estudiantes de esta fabulosa rama de la Inteligencia Artificial.